In [ ]:
# ruff: noqa

# Quick Start

See the [API](https://xskillscore.readthedocs.io/en/stable/api.html) for more detailed information, examples, formulas, and references for each function.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import xskillscore as xs

np.random.seed(seed=42)

Here, we generate some sample gridded data. Our data has three time steps, and a 4x5 latitude/longitude grid. `obs` denotes some verification data (sometimes termed `y`) and `fct` some forecast data (e.g. from a statistical or dynamical model; sometimes termed `yhat`).

In [ ]:
obs = xr.DataArray(
    np.random.rand(3, 4, 5),
    coords=[
        xr.cftime_range("2000-01-01", "2000-01-03", freq="D"),
        np.arange(4),
        np.arange(5),
    ],
    dims=["time", "lat", "lon"],
    name="var",
)
fct = obs.copy()
fct.values = np.random.rand(3, 4, 5)

## Deterministic Metrics

`xskillscore` offers a suite of correlation-based and distance-based deterministic metrics:

### Correlation-Based 

* Effective Sample Size (`effective_sample_size`)
* Pearson Correlation (`pearson_r`)
* Pearson Correlation effective p value (`pearson_r_eff_p_value`)
* Pearson Correlation p value (`pearson_r_p_value`)
* Slope of Linear Fit (`linslope`)
* Spearman Correlation (`spearman_r`)
* Spearman Correlation effective p value (`spearman_r_eff_p_value`)
* Spearman Correlation p value (`spearman_r_p_value`)

### Distance-Based

* Coefficient of Determination (`r2`)
* Mean Absolute Error (`mae`)
* Mean Absolute Percentage Error (`mape`)
* Mean Error (`me`)
* Mean Squared Error (`mse`)
* Median Absolute Error (`median_absolute_error`)
* Root Mean Squared Error (`rmse`)
* Symmetric Mean Absolute Percentage Error (`smape`)

Calling the functions is very straight-forward. All deterministic functions take the form `func(a, b, dim=None, **kwargs)`. **Notice that the original dataset is reduced by the dimension passed.** I.e., since we passed `time` as the dimension here, we are returned an object with dimensions `(lat, lon)`. For correlation metrics `dim` cannot be `[]`.

In [ ]:
r = xs.pearson_r(obs, fct, dim="time")
print(r)

In [ ]:
p = xs.pearson_r_p_value(obs, fct, dim="time")
print(p)

You can also specify multiple axes for deterministic metrics. Here, we apply it over the latitude and longitude dimension (a pattern correlation).

In [ ]:
r = xs.pearson_r(obs, fct, dim=["lat", "lon"])
print(r)

All deterministic metrics except for `effective_sample_size`, `pearson_r_eff_p_value` and `spearman_r_eff_p_value` can take the kwarg `weights=...`. `weights` should be a DataArray of the size of the reduced dimension (e.g., if time is being reduced it should be of length 3 in our example).

Weighting is a common practice when working with observations and model simulations of the Earth system. When working with rectilinear grids, one can weight the data by the cosine of the latitude, which is maximum at the equator and minimum at the poles (as in the below example). More complicated model grids tend to be accompanied by a cell area varaible, which could also be passed into this function.

In [ ]:
obs2 = xr.DataArray(
    np.random.rand(3, 180, 360),
    coords=[
        xr.cftime_range("2000-01-01", "2000-01-03", freq="D"),
        np.linspace(-89.5, 89.5, 180),
        np.linspace(-179.5, 179.5, 360),
    ],
    dims=["time", "lat", "lon"],
)
fct2 = obs2.copy()
fct2.values = np.random.rand(3, 180, 360)

In [ ]:
# make weights as cosine of the latitude and broadcast
weights = np.cos(np.deg2rad(obs2.lat))
_, weights = xr.broadcast(obs2, weights)

# Remove the time dimension from weights
weights = weights.isel(time=0)

In [ ]:
r_weighted = xs.pearson_r(obs2, fct2, dim=["lat", "lon"], weights=weights)
print(r_weighted)

In [ ]:
r_unweighted = xs.pearson_r(obs2, fct2, dim=["lat", "lon"], weights=None)
print(r_unweighted)

You can also pass the optional boolean kwarg `skipna`. If `True`, ignore any NaNs (pairwise) in `obs` and `fct` when computing the result. If `False`, return NaNs anywhere there are pairwise NaNs.

In [ ]:
obs_with_nans = obs.where(obs.lat > 1)
fct_with_nans = fct.where(fct.lat > 1)
print(obs_with_nans)

In [ ]:
mae_with_skipna = xs.mae(obs_with_nans, fct_with_nans, dim=["lat", "lon"], skipna=True)
print(mae_with_skipna)

In [ ]:
mae_without_skipna = xs.mae(
    obs_with_nans, fct_with_nans, dim=["lat", "lon"], skipna=False
)
print(mae_without_skipna)

## Probabilistic Metrics

`xskillscore` offers a suite of probabilistic metrics:

* Brier Score (`brier_score`)
* Brier scores of an ensemble for exceeding given thresholds (`threshold_brier_score`)
* Continuous Ranked Probability Score with a gaussian distribution (`crps_gaussian`)
* Continuous Ranked Probability Score with numerical integration of the normal distribution (`crps_quadrature`)
* Continuous Ranked Probability Score with the ensemble distribution (`crps_ensemble`)
* Discrimination (`discrimination`)
* Rank Histogram (`rank_histogram`)
* Ranked Probability Score (`rps`)
* Receiver Operating Characteristic (`roc`)
* Reliability (`reliability`)

We now create some data with an ensemble member dimension. In this case, we envision an ensemble forecast with multiple members to validate against our theoretical observations:

In [ ]:
obs3 = xr.DataArray(
    np.random.rand(4, 5),
    coords=[np.arange(4), np.arange(5)],
    dims=["lat", "lon"],
    name="var",
)
fct3 = xr.DataArray(
    np.random.rand(3, 4, 5),
    coords=[np.arange(3), np.arange(4), np.arange(5)],
    dims=["member", "lat", "lon"],
    name="var",
)

Continuous Ranked Probability Score with the ensemble distribution. Pass `dim=[]` to get the same behaviour as `properscoring.crps_ensemble` without any averaging over `dim`.

In [ ]:
crps_ensemble = xs.crps_ensemble(obs3, fct3, dim=[])
print(crps_ensemble)

The CRPS with a Gaussian distribution requires two parameters: $\mu$ and $\sigma$ from the forecast distribution. Here, we just use the ensemble mean and ensemble spread.

In [ ]:
crps_gaussian = xs.crps_gaussian(obs3, fct3.mean("member"), fct3.std("member"), dim=[])
print(crps_gaussian)

The CRPS quadrature metric requires a callable distribution function. Here we use `norm` from `scipy.stats`.

In [ ]:
from scipy.stats import norm

crps_quadrature = xs.crps_quadrature(obs3, norm, dim=[])
print(crps_quadrature)

We can also use a threshold Brier Score, to score hits over a certain threshold. Ranked Probability Score for two categories yields the same result.

In [ ]:
threshold_brier_score = xs.threshold_brier_score(obs3, fct3, 0.5, dim=None)
print(threshold_brier_score)

In [ ]:
brier_score = xs.brier_score(obs3 > 0.5, (fct3 > 0.5).mean("member"))
print(brier_score)

In [ ]:
rps = xs.rps(obs3 > 0.5, fct3 > 0.5, category_edges=np.array([0.5]))
print(rps)

In [ ]:
rank_histogram = xs.rank_histogram(obs3, fct3)
print(rank_histogram)

In [ ]:
disc = xs.discrimination(obs3 > 0.5, (fct3 > 0.5).mean("member"))
print(disc)

In [ ]:
rel = xs.reliability(obs3 > 0.5, (fct3 > 0.5).mean("member"))
print(rel)

In [ ]:
# ROC for probabilistic forecasts and bin_edges='continuous' default
roc = xs.roc(
    obs3 > 0.5, (fct3 > 0.5).mean("member"), return_results="all_as_metric_dim"
)

plt.figure(figsize=(4, 4))
plt.plot([0, 1], [0, 1], "k:")
roc.to_dataset(dim="metric").plot.scatter(
    y="true positive rate", x="false positive rate"
)
roc.sel(metric="area under curve").values[0]

## Contingency-Based

To work with contingency-based scoring, first instantiate a `Contingency` object by passing in your observations, forecast, and observation/forecast bin edges. See https://www.cawcr.gov.au/projects/verification/#Contingency_table for more information.

In [ ]:
dichotomous_category_edges = np.array([0, 0.5, 1])  # "dichotomous" mean two-category
dichotomous_contingency = xs.Contingency(
    obs, fct, dichotomous_category_edges, dichotomous_category_edges, dim=["lat", "lon"]
)
dichotomous_contingency_table = dichotomous_contingency.table
print(dichotomous_contingency_table)

In [ ]:
print(
    dichotomous_contingency_table.to_dataframe()
    .pivot_table(
        index=["forecasts_category", "forecasts_category_bounds"],
        columns=["observations_category", "observations_category_bounds"],
    )
    .round(2)
)

Scores based on the constructed contingency table can be called via class methods. The available methods are:

* Accuracy (`accuracy`)
* Bias Score (`bias_score`)
* Equitable Threat Score (`equit_threat_score`)
* False Alarms / False Positives (`false_alarms`)
* False Alarm Ratio / False Discovery Rate (`false_alarm_ratio`)
* False Alarm Rate / False Positive Rate / Fall-out (`false_alarm_rate`)
* Gerrity Score (`gerrity_score`)
* Heidke Score / Cohen's Kappa (`heidke_score`)
* Hit Rate / Recall / Sensitivity / True Positive Rate (`hit_rate`)
* Hits / True Positives (`hits`)
* Misses / False Negatives (`misses`)
* Odds Ratio (`odds_ratio`)
* Odds Ratio Skill Score (`odds_ratio_skill_score`)
* Peirce Score (`peirce_score`)
* Receiver Operating Characteristic (`roc`)
* Success Ratio / Precision / Positive Predictive Value (`success_ratio`)
* Threat Score / Critical Success Index (`threat_score`)

Below, we share a few examples of these in action:

In [ ]:
print(dichotomous_contingency.bias_score())

In [ ]:
print(dichotomous_contingency.hit_rate())

In [ ]:
print(dichotomous_contingency.false_alarm_rate())

In [ ]:
print(dichotomous_contingency.odds_ratio_skill_score())

Now we can leverage multi-category edges to make use of some scores.

In [ ]:
multi_category_edges = np.array([0, 0.25, 0.75, 1])
multicategory_contingency = xs.Contingency(
    obs, fct, multi_category_edges, multi_category_edges, dim=["lat", "lon"]
)

In [ ]:
print(multicategory_contingency.accuracy())

In [ ]:
print(multicategory_contingency.heidke_score())

In [ ]:
print(multicategory_contingency.peirce_score())

In [ ]:
print(multicategory_contingency.gerrity_score())

In [ ]:
# ROC for deterministic forecasts and bin_edges
roc = xs.roc(obs, fct, np.linspace(0, 1, 11), return_results="all_as_metric_dim")

plt.figure(figsize=(4, 4))
plt.plot([0, 1], [0, 1], "k:")
roc.to_dataset(dim="metric").plot.scatter(
    y="true positive rate", x="false positive rate"
)
roc.sel(metric="area under curve").values[0]

## Comparative

Tests to compare whether one forecast is significantly better than another one.

### Sign test

In [ ]:
length = 100
obs_1d = xr.DataArray(
    np.random.rand(length),
    coords=[
        np.arange(length),
    ],
    dims=["time"],
    name="var",
)
fct_1d = obs_1d.copy()
fct_1d.values = np.random.rand(length)

In [ ]:
# given you want to test whether one forecast is better than another forecast
significantly_different, walk, confidence = xs.sign_test(
    fct_1d, fct_1d + 0.2, obs_1d, time_dim="time", metric="mae", orientation="negative"
)

In [ ]:
walk.plot()
confidence.plot(c="gray")
(-1 * confidence).plot(c="gray")

### Half-width Confidence Interval test

In [ ]:
# create a worse forecast with high but different to perfect correlation and choose RMSE as the distance metric
fct_1d_worse = fct_1d.copy()
step = 3
fct_1d_worse[::step] = fct_1d[::step].values + 0.1
metric = "rmse"

In [ ]:
# half-with of the confidence interval at level alpha is larger than the RMSE differences,
# therefore not significant
alpha = 0.05
significantly_different, diff, hwci = xs.halfwidth_ci_test(
    fct_1d, fct_1d_worse, obs_1d, metric, time_dim="time", dim=[], alpha=alpha
)
print(diff)
print(hwci)
print(
    f"RMSEs significantly different at level {alpha} : {bool(significantly_different)}"
)

## Statistical tests

### Multiple tests

A statistical test is applied multiple times on a spatial grid. `multipletests` helps controlling the false discovery rate (FDR) of p values.

In [ ]:
p = xs.pearson_r_p_value(fct, obs, "time")
print(p)

In [ ]:
p_corrected = xs.multipletests(
    p, alpha=0.5, method="fdr_bh", return_results="pvals_corrected"
)
print(p_corrected)

In [ ]:
assert not p_corrected.equals(p)

## Accessors

You can also use `xskillscore` as a method of your `xarray` Dataset.

In [ ]:
ds = xr.Dataset()
ds["obs_var"] = obs
ds["fct_var"] = fct

In the case that your Dataset contains both your observation and forecast variable, just pass them as strings into the function.

In [ ]:
print(ds.xs.pearson_r("obs_var", "fct_var", dim="time"))

You can also pass in a separate Dataset that contains your observations or forecast variable.

In [ ]:
ds = ds.drop_vars("fct_var")
print(ds.xs.pearson_r("obs_var", fct, dim="time"))

## Resampling
-   randomly resample the `time` dimension and then take mean over `time` to get resample threshold
-   resample over `member` dimension to get uncertainty due to member sampling in hindcasts

In [ ]:
# create large one-dimensional array
s = 1000
f = xr.DataArray(
    np.random.normal(size=s), dims="member", coords={"member": np.arange(s)}, name="var"
)

In [ ]:
# resample with replacement in that one dimension
iterations = 100
%timeit f_r = xs.resampling.resample_iterations(f, iterations, 'member', replace=True)
# resample_iterations_idx is much (50x) faster because it involves no loops
%timeit f_r = xs.resampling.resample_iterations_idx(f, iterations, 'member', replace=True)
# but both do the same resampling

- use `resample_iterations` for very large data, because very robust, chunksize stays contants and only more tasks are added
- use `resample_iterations_idx` for small data always and very large data only, when chunked to small chunks in the other dimensions, because the function increases the input chunksize by factor `iterations`

In [ ]:
f_r = xs.resampling.resample_iterations_idx(f, iterations, "member", replace=True)
f.plot.hist(label="distribution")
f_r.mean("iteration").plot.hist(label="resampled mean distribution")
plt.axvline(x=f.mean("member"), c="k", label="distribution mean")
plt.title("Gaussian distribution mean")
plt.legend()

In [ ]:
# we can calculate the distribution of the RMSE of 0 and f resampled over member
xs.rmse(f_r, xr.zeros_like(f_r), dim="iteration").plot.hist(
    label="resampled RMSE distribution"
)
# the gaussian distribution should have an RMSE with 0 of one
plt.axvline(x=xs.rmse(f, xr.zeros_like(f)), c="k", label="RMSE")
plt.title("RMSE between gaussian distribution and 0")
plt.legend()